In [11]:
#Find and print the details of all books which are present on first 2 pages of this website.
#All details include - Title of the book, book page url, Price (in float, without any currency or extra symbol), and quantity in stock (in integer). Save all the details in a dataframe and print in the required format.
main_url = "http://books.toscrape.com/index.html"
allPages = ['http://books.toscrape.com/catalogue/page-1.html',
            'http://books.toscrape.com/catalogue/page-2.html']


import requests
result = requests.get(main_url)

from bs4 import BeautifulSoup
soup = BeautifulSoup(result.text, 'html.parser')


#Let’s define a function to request and parse a HTML web page as we will need this a lot
def getAndParseURL(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)

def getBooksURLs(url):
    soup = getAndParseURL(url)
    # remove the index.html part of the base url before returning the results
    return(["/".join(url.split("/")[:-1]) + "/" + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")])

import re

categories_urls = [main_url + x.get('href') for x in soup.find_all("a", href=re.compile("catalogue/category/books"))]
categories_urls = categories_urls[1:]

booksURLs = []
for page in allPages:
    booksURLs.extend(getBooksURLs(page))


names = []
links = []
prices = []
nb_in_stock = []
# img_urls = []
# categories = []
# ratings = []
# column_names = ['Title', 'Link', 'Price', 'Quantity in Stock']


# scrape data for every book URL: this may take some time
for url in booksURLs:
    soup = getAndParseURL(url)
    
    # product name
    names.append(soup.find("div", class_ = re.compile("product_main")).h1.text)
    
    #book link
    links.append(url)
    
    # product price
    prices.append(soup.find("p", class_ = "price_color").text[:]) # get rid of the pound sign
    
    # number of available products
    nb_in_stock.append(re.sub("[^0-9]", "", soup.find("p", class_ = "instock availability").text)) # get rid of non numerical characters
    
#     # image url
#     img_urls.append(url.replace("index.html", "") + soup.find("img").get("src"))
    
#     # product category
#     categories.append(soup.find("a", href = re.compile("../category/books/")).get("href").split("/")[3])
    
#     # ratings
#     ratings.append(soup.find("p", class_ = re.compile("star-rating")).get("class")[1])
    
# add data into pandas df
import pandas as pd

scraped_data = pd.DataFrame({'Title': names,'Link': links,'Price': prices,'Quantity in Stock': nb_in_stock})# "url_img": img_urls, "product_category": categories, "rating": ratings})
scraped_data.style.hide_index()

Title,Link,Price,Quantity in Stock
A Light in the Attic,http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html,51.77,22
Tipping the Velvet,http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html,53.74,20
Soumission,http://books.toscrape.com/catalogue/soumission_998/index.html,50.10,20
Sharp Objects,http://books.toscrape.com/catalogue/sharp-objects_997/index.html,47.82,20
Sapiens: A Brief History of Humankind,http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html,54.23,20
The Requiem Red,http://books.toscrape.com/catalogue/the-requiem-red_995/index.html,22.65,19
The Dirty Little Secrets of Getting Your Dream Job,http://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html,33.34,19
"The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull",http://books.toscrape.com/catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html,17.93,19
The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics,http://books.toscrape.com/catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html,22.60,19
The Black Maria,http://books.toscrape.com/catalogue/the-black-maria_991/index.html,52.15,19
